In [1]:
import tensorflow as tf

In [2]:
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

In [3]:
class Conv2D(tf.keras.layers.Conv2D):
    def __init__(self,f,k,a,b,**kwdarg):
        super().__init__(
            filters=f,
            kernel_size=k,
            activation=a,
            use_bias=b,
            strides=1,
            padding='valid',
            **kwdarg)
    def call(self,inp,training):
        x = super().call(inp)
        return x

In [4]:
class DepthwiseConv2D(tf.keras.layers.DepthwiseConv2D):
    def __init__(self,m,k,a,b,**kwdarg):
        super().__init__(
            depth_multiplier=m,
            kernel_size=k,
            activation=a,
            use_bias=b,
            strides=1,
            padding='valid',
            **kwdarg)
    def call(self,inp,training):
        x = super().call(inp)
        return x

In [5]:
class DepthControl(tf.keras.layers.Layer):
    def __init__(self,r,a,b):
        super().__init__()
        self.r = 2**r
        self.a = a
        self.b = b
    def build(self,input_shape):
        self.conv = Conv2D(
            f=int(input_shape[-1]*self.r),
            k=1,
            a=self.a,
            b=self.b)
    def call(self,inp,training):
        x = self.conv(inp,training=training)
        return x

In [6]:
class DownSample(tf.keras.layers.Layer):
    def __init__(self,k=(2,2)):
        super().__init__()
        self.k = k
    def build(self,input_shape):
        self.output_size = (input_shape[1]//self.k[0],input_shape[2]//self.k[1])
    def call(self,inp,training):
        return tf.image.resize(inp, self.output_size)

In [7]:
class Contract(tf.keras.layers.Layer):
    def __init__(self,k):
        super().__init__()
        self.downsample = DownSample(k)
        self.increase = DepthControl(r=1,a=None,b=False)
    def call(self,inp,training):
        x = self.downsample(inp,training=training)
        x = self.increase(x,training=training)
        return x

In [8]:
class Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.block1 = tf.keras.Sequential([
            DepthwiseConv2D(m=32,k=3,a=None,b=False),
            Conv2D(f=32,k=3,a=mish,b=True),
            Contract((1,2))
        ])
        self.block2 = tf.keras.Sequential([
            Conv2D(f=64,k=3,a=mish,b=True),
            Contract((1,2))
        ])
        self.block3 = tf.keras.Sequential([
            Conv2D(f=128,k=3,a=mish,b=True),
            Contract((1,2))
        ])
        self.block4 = tf.keras.Sequential([
            Conv2D(f=256,k=3,a=mish,b=True),
            Contract((2,2))
        ])
        self.block5 = tf.keras.Sequential([
            Conv2D(f=512,k=2,a=mish,b=True),
            Contract((1,2))
        ])
        self.block6 = tf.keras.Sequential([
            Conv2D(f=512,k=3,a=mish,b=True),
            Conv2D(f=512,k=3,a=mish,b=True),
            Conv2D(f=512,k=2,a=mish,b=True)
        ])
        self.output_layer = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1024,activation=mish,use_bias=True),
            tf.keras.layers.Dense(512,activation=mish,use_bias=True),
            tf.keras.layers.Dense(256,activation=mish,use_bias=True),
            tf.keras.layers.Dense(128,activation=mish,use_bias=True),
            tf.keras.layers.Dense(64,activation=mish,use_bias=True),
            tf.keras.layers.Dense(32,activation=mish,use_bias=True),
            tf.keras.layers.Dense(16,activation=mish,use_bias=True),
            tf.keras.layers.Dense(4,activation=mish,use_bias=True),
            tf.keras.layers.Dense(1,activation='sigmoid',use_bias=False),
        ])
        
    def call(self,image,training):
        x = self.block1(image,training=training)
        x = self.block2(x,training=training)
        x = self.block3(x,training=training)
        x = self.block4(x,training=training)
        x = self.block5(x,training=training)
        x = self.block6(x,training=training)
        x = self.output_layer(x,training=training)
        return x
    
    def predict(self,image):
        predict = self(image,training=False)
        predict = tf.squeeze(predict,axis=-1)
        predict = predict.numpy()
        predict = predict > 0.5
        return predict

In [9]:
model = Model()

In [10]:
# %load_ext tensorboard
# %tensorboard --logdir=logs

In [11]:
import FLAGS

FLAGS.DATA.TRAIN.DATA_PATH = r'D:\Competitions\ComputerVision\OCR\ChinaSteel\dataset\train\segment\image_manually_classified_by_overturn'
FLAGS.DATA.TRAIN.VALIDATION_SPLIT_RATIO = 0.05
FLAGS.DATA.TRAIN.VALIDATION_SPLIT_RANDOM_STATE = 100
FLAGS.DATA.TRAIN.TRAIN_BATCH_SIZE = 8
FLAGS.DATA.TRAIN.VAL_BATCH_SIZE = 8
FLAGS.DATA.TEST.IMAGE_PATH = r'D:\Competitions\ComputerVision\OCR\ChinaSteel\dataset\test\segment\sample'
FLAGS.DATA.TEST.BATCH_SIZE = 8

FLAGS.MODEL = model

FLAGS.LOSS.LABEL_SMOOTHING = 0.001

FLAGS.OPTIMIZER.TYPE =  tf.keras.optimizers.Adam
FLAGS.OPTIMIZER.MAX_LEARNING_RATE = 0.5*1e-3
FLAGS.OPTIMIZER.SCHEDULE_GAMMA = -1.5

FLAGS.EPOCHS.TOTAL = 100
FLAGS.EPOCHS.WARMUP = 2

FLAGS.LOGGING.PATH = 'logs'
FLAGS.LOGGING.MODEL_NAME = 1
FLAGS.LOGGING.TRIAL_NUMBER = 1
FLAGS.LOGGING.NOTE = 'None'
FLAGS.LOGGING.SAMPLES_PER_LOG = 128

In [12]:
%run train.py

KeyboardInterrupt: 